# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random

from extrucal.extrusion import throughput_cal

random.seed(0)

# Dataset Generation using `extrucal`

### 1. Preparation of Lists for Arguments

In [2]:
extruder_size = [size for size in range(20, 251, 10)]
extruder_size_df = pd.DataFrame({"extruder_size": extruder_size})
# random.shuffle(extruder_size)    # shuffle works in place and return None
len(extruder_size)

24

In [3]:
metering_depth_percent = [np.round(percent, 2) for percent in np.arange(0.02, 0.1, 0.01)]
metering_depth_percent_df = pd.DataFrame({"metering_depth_percent": metering_depth_percent})
# random.shuffle(metering_depth_percent)
len(metering_depth_percent)

8

In [4]:
polymer_density = [density for density in range(800, 1500, 100)]
polymer_density_df = pd.DataFrame({"polymer_density": polymer_density})
# random.shuffle(polymer_density)
len(polymer_density)

7

In [5]:
screw_pitch_percent = [np.round(pitch, 2) for pitch in np.arange(0.6, 2, 0.2)]
screw_pitch_percent_df = pd.DataFrame({"screw_pitch_percent": screw_pitch_percent})
# random.shuffle(screw_pitch_percent)
len(screw_pitch_percent)

7

In [6]:
flight_width_percent = [np.round(percent, 2) for percent in np.arange(0.06, 0.2, 0.02)]
flight_width_percent_df = pd.DataFrame({"flight_width_percent": flight_width_percent})
# random.shuffle(flight_width_percent)
len(flight_width_percent)

8

In [7]:
number_flight = [1, 2]
number_flight_df = pd.DataFrame({"number_flight": number_flight})
# random.shuffle(number_flight)
len(number_flight)

2

In [8]:
rpm = [r for r in range(0, 100, 10)]
rpm_df = pd.DataFrame({"rpm": rpm})
# random.shuffle(rpm)
len(rpm)

10

### 2. Preparation of Dataframe

In [9]:
df = (extruder_size_df
      .join(metering_depth_percent_df, how='cross')
      .join(polymer_density_df, how='cross')
      .join(screw_pitch_percent_df, how='cross')
      .join(flight_width_percent_df, how='cross')
      .join(number_flight_df, how='cross')
      .join(rpm_df, how='cross'))

In [10]:
len(df)

1505280

In [11]:
df["metering_depth"] = df["extruder_size"] * df["metering_depth_percent"]
df["screw_pitch"] = df["extruder_size"] * df["screw_pitch_percent"]
df["flight_width"] = df["extruder_size"] * df["flight_width_percent"]

In [12]:
new_col_order = [
    "extruder_size", "metering_depth", "polymer_density", 
    "rpm", "screw_pitch", "flight_width", "number_flight", ]

df = df[new_col_order]
df.head()

,extruder_size,metering_depth,polymer_density,rpm,screw_pitch,flight_width,number_flight
0,20,0.4,800,0,12.0,1.2,1
1,20,0.4,800,10,12.0,1.2,1
2,20,0.4,800,20,12.0,1.2,1
3,20,0.4,800,30,12.0,1.2,1
4,20,0.4,800,40,12.0,1.2,1


In [13]:
%%time

df.loc[:, "throughput"] = df.apply(
    lambda row: throughput_cal(
        row["extruder_size"],
        row["metering_depth"],
        row["polymer_density"],
        row["rpm"],
        row["screw_pitch"],
        row["flight_width"],
        int(row["number_flight"])), axis=1
)

CPU times: total: 9min 51s
Wall time: 9min 52s


In [14]:
df

,extruder_size,metering_depth,polymer_density,rpm,screw_pitch,flight_width,number_flight,throughput
0,20,0.4,800,0,12.0,1.2,1,0.00
1,20,0.4,800,10,12.0,1.2,1,0.06
2,20,0.4,800,20,12.0,1.2,1,0.12
3,20,0.4,800,30,12.0,1.2,1,0.18
4,20,0.4,800,40,12.0,1.2,1,0.24
...,...,...,...,...,...,...,...,...
1505275,250,22.5,1400,50,450.0,50.0,2,8207.87
1505276,250,22.5,1400,60,450.0,50.0,2,9849.44
1505277,250,22.5,1400,70,450.0,50.0,2,11491.02
1505278,250,22.5,1400,80,450.0,50.0,2,13132.59


### 3. Save as CSV file

In [15]:
df.to_csv("../data/extrucal_dataset_improved.csv", index=False)